<a href="https://colab.research.google.com/github/mkm-world/laduma-analytics/blob/main/zindi_football_baseline_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Zindi Weekendz: Laduma Analytics Football League Winners Prediction Challenge

- **Can you predict the outcome of a football match based on historical data?**

# Downloading all datasets directly from Zindi platform using url:

#### Import Libraries

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime as dt
import itertools
from tqdm import tqdm
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

#Load Datasets:

In [2]:
path = '../input/laduma/'

In [3]:
train = pd.read_csv(path+"Train.csv",parse_dates=['Date'])
test = pd.read_csv(path+"Test.csv",parse_dates=['Date'])
train_stats = pd.read_csv('../input/laduma/train_game_statistics.csv')
test_stats = pd.read_csv('../input/laduma/test_game_statistics.csv')

In [44]:
def mean_X_change(stats):
    
    ch = stats.groupby(['Game_ID','Team']).apply(lambda x: x.Player_ID.unique().shape[0]%11  )
    first_appear = stats.groupby(['Game_ID','Team','Player_ID']).apply(lambda x: x['Start_minutes'].min())
    mean_X = stats.groupby(['Game_ID','Team','Player_ID']).apply(lambda x:  x['X'].mean())
    new_df = first_appear.reset_index().merge(mean_X.reset_index(),on = ['Game_ID','Team','Player_ID'],how = 'left')
    new_df = new_df.groupby(['Game_ID','Team']).apply(lambda x : x.sort_values(by = '0_x',ascending = False)).reset_index(drop = True)
    df = ch.reset_index()[['Game_ID','Team']].copy()
    df[['mean_change_X' ]] =np.nan
    ch = ch.reset_index()
    for g in tqdm(ch.Game_ID.unique()):
        for t in ch.loc[ch.Game_ID == g,'Team']:
            i = ch.loc[(ch.Game_ID == g)&(ch.Team == t),0].values[0]
            df.loc[(df.Game_ID == g)&(df.Team == t),'mean_change_X'] = np.mean(new_df.loc[(new_df.Game_ID == g)&(new_df.Team == t),'0_y'].values[:i])
    
    return df
def preprocess(train,test,train_stats,test_stats,scale = False,th = 2):
    train['Train']=1
    test['Train']=0
    home_teams=train['Home Team'].unique()
    all_data=pd.concat([train,test])
    for team in home_teams:
        all_data['home_'+team]=0
        all_data.loc[all_data['Home Team']==team,'home_'+team]=1
    away_teams=train['Away Team'].unique()
    for team in away_teams:
        all_data['away_'+team]=0
        all_data.loc[all_data['Away Team']==team,'away_'+team]=1
#     print(all_data.isna().sum())
    all_data.fillna(all_data.mean(),inplace=True)
    all_data['month'] = all_data.Date.dt.month
    all_data['year'] = all_data.Date.dt.year
    all_data['day'] = all_data.Date.dt.day
    all_data['week'] = all_data.Date.dt.week
    all_data['dow'] = all_data.Date.dt.dayofweek
    all_data['woy'] = all_data.Date.dt.weekofyear
    train = all_data[all_data["Train"] == 1]
    test = all_data[all_data["Train"] == 0]
    train_inv= train.copy()
    train_inv['Home Team'] = train['Away Team']
    train_inv['Away Team'] = train['Home Team']
    train['last_Score'] = 0
    test['last_Score'] = 0
    for game in tqdm(train.Game_ID):
        teamh = train.loc[train.Game_ID == game,'Home Team'].values[0]
        teama = train.loc[train.Game_ID == game,'Away Team'].values[0]
        date = train.loc[train.Game_ID == game,'Date'].values[0]
        seas = train.loc[train.Game_ID == game,'Season'].values[0]
        temp = train.loc[(train['Home Team'] == teamh) & (train['Away Team'] == teama),
                         ['Game_ID','Date','Season','Score'] ]
        temp_inv = train_inv.loc[(train_inv['Home Team'] == teamh) & (train_inv['Away Team'] == teama),
                         ['Game_ID','Date','Season','Score']]
        temp_inv['Score'] = temp_inv['Score'].map({0:2,2:0})
        if (temp.shape[0] ==0) & (temp_inv.shape[0] ==0):
            train.loc[train.Game_ID == game,'last_Score'] = -1 # No data
        elif (temp.shape[0] ==0):
            game_id = -1
            for g in temp_inv.Game_ID:
                if temp_inv.loc[temp_inv.Game_ID == g,'Date'].values[0]<date:
                    game_id = g
            if game_id == -1:
                train.loc[train.Game_ID == game,'last_Score'] = -1 # No recent data
            else:
                train.loc[train.Game_ID == game,'last_Score'] = temp_inv.loc[temp_inv.Game_ID == game_id,'Score'].values[0]
        elif (temp_inv.shape[0] ==0):
            game_id = -1
            for g in temp.Game_ID:
                if temp.loc[temp.Game_ID == g,'Date'].values[0]<date:
                    game_id = g
            if game_id == -1:
                train.loc[train.Game_ID == game,'last_Score'] = -1 # No recent data
            else:
                train.loc[train.Game_ID == game,'last_Score'] = temp.loc[temp.Game_ID == game_id,'Score'].values[0]
        else:
            game_id_inv = -1
            for g in temp_inv.Game_ID:
                if temp_inv.loc[temp_inv.Game_ID == g,'Date'].values[0]<date:
                    game_id_inv = g
            game_id = -1
            for g in temp.Game_ID:
                if temp.loc[temp.Game_ID == g,'Date'].values[0]<date:
                    game_id = g
            if (game_id == -1) & (game_id_inv == -1):
                train.loc[train.Game_ID == game,'last_Score'] = -1 # No data
            elif game_id ==-1:
                train.loc[train.Game_ID == game,'last_Score'] = temp_inv.loc[temp_inv.Game_ID == game_id_inv,'Score'].values[0]
            elif game_id_inv == -1:
                train.loc[train.Game_ID == game,'last_Score'] = temp.loc[temp.Game_ID == game_id,'Score'].values[0]
            else:
                if temp.loc[temp.Game_ID == game_id,'Date'].values[0]> temp_inv.loc[temp_inv.Game_ID == game_id_inv,'Date'].values[0]:
                    train.loc[train.Game_ID == game,'last_Score'] = temp_inv.loc[temp_inv.Game_ID == game_id_inv,'Score'].values[0]
                else:
                    train.loc[train.Game_ID == game,'last_Score'] = temp.loc[temp.Game_ID == game_id,'Score'].values[0]
    for game in tqdm(test.Game_ID):
        teamh = test.loc[test.Game_ID == game,'Home Team'].values[0]
        teama = test.loc[test.Game_ID == game,'Away Team'].values[0]
        date = test.loc[test.Game_ID == game,'Date'].values[0]
        seas = test.loc[test.Game_ID == game,'Season'].values[0]
        temp = train.loc[(train['Home Team'] == teamh) & (train['Away Team'] == teama),
                         ['Game_ID','Date','Season','Score'] ]
        temp_inv = train_inv.loc[(train_inv['Home Team'] == teamh) & (train_inv['Away Team'] == teama),
                         ['Game_ID','Date','Season','Score']]
        temp_inv['Score'] = temp_inv['Score'].map({0:2,2:0})
        if (temp.shape[0] ==0) & (temp_inv.shape[0] ==0):
            test.loc[test.Game_ID == game,'last_Score'] = np.nan # No data
        elif (temp.shape[0] ==0):
            game_id = None
            for g in temp_inv.Game_ID:
                if temp_inv.loc[temp_inv.Game_ID == g,'Date'].values[0]<date:
                    game_id = g
            if game_id == None:
                test.loc[test.Game_ID == game,'last_Score'] = np.nan # No recent data
            else:
                test.loc[test.Game_ID == game,'last_Score'] = temp_inv.loc[temp_inv.Game_ID == game_id,'Score'].values[0]
        elif (temp_inv.shape[0] ==0):
            game_id = None
            for g in temp.Game_ID:
                if temp.loc[temp.Game_ID == g,'Date'].values[0]<date:
                    game_id = g
            if game_id == None:
                test.loc[test.Game_ID == game,'last_Score'] = np.nan # No recent data
            else:
                test.loc[test.Game_ID == game,'last_Score'] = temp.loc[temp.Game_ID == game_id,'Score'].values[0]
        else:
            game_id_inv = None
            for g in temp_inv.Game_ID:
                if temp_inv.loc[temp_inv.Game_ID == g,'Date'].values[0]<date:
                    game_id_inv = g
            game_id = None
            for g in temp.Game_ID:
                if temp.loc[temp.Game_ID == g,'Date'].values[0]<date:
                    game_id = g
            if (game_id == None) & (game_id_inv == None):
                test.loc[test.Game_ID == game,'last_Score'] = np.nan # No data
            elif game_id ==None:
                test.loc[test.Game_ID == game,'last_Score'] = temp_inv.loc[temp_inv.Game_ID == game_id_inv,'Score'].values[0]
            elif game_id_inv == None:
                test.loc[test.Game_ID == game,'last_Score'] = temp.loc[temp.Game_ID == game_id,'Score'].values[0]
            else:
                if temp.loc[temp.Game_ID == game_id,'Date'].values[0]> temp_inv.loc[temp_inv.Game_ID == game_id_inv,'Date'].values[0]:
                    test.loc[test.Game_ID == game,'last_Score'] = temp_inv.loc[temp_inv.Game_ID == game_id_inv,'Score'].values[0]
                else:
                    test.loc[test.Game_ID == game,'last_Score'] = temp.loc[temp.Game_ID == game_id,'Score'].values[0]
    all_stats = pd.concat([train_stats,test_stats])
    all_stats.drop(['Action', 'Goals_scored', 'Goals_conceded', 'next_action'],axis=1,inplace = True)
    all_stats.drop(['next_player','next_x','next_y','event_id','next_team','next_event_id','xt_value'],axis =1,inplace = True)
    # use grouped to calculate other statistics
    all_stats = all_stats[all_stats.X !=300]
    grouped = all_stats.groupby(['Game_ID','Team','Player_ID','Half'])[['Shots','SoT','Accurate passes','Inaccurate passes','Passes']].sum()
    grouped =grouped.reset_index()
    accu_shots = grouped.groupby(['Game_ID','Team','Half']).apply(lambda x : x['SoT'].sum()/x['Shots'].sum()).reset_index()
    accu_pass = grouped.groupby(['Game_ID','Team','Half']).apply(lambda x : x['Accurate passes'].sum()/x['Passes'].sum()).reset_index()
    n_shots = grouped.groupby(['Game_ID','Team','Half']).apply(lambda x : x['SoT'].sum()).reset_index()
    n_passes = grouped.groupby(['Game_ID','Team','Half']).apply(lambda x : x['Passes'].sum()).reset_index()
    train[['n_passes_home1','n_shots_home1','accu_pass_home1','accu_shots_home1',
          'n_passes_away1','n_shots_away1','accu_pass_away1','accu_shots_away1',
          'n_passes_home2','n_shots_home2','accu_pass_home2','accu_shots_home2',
          'n_passes_away2','n_shots_away2','accu_pass_away2','accu_shots_away2']]=0
    for game in tqdm(train.Game_ID):
        train.loc[train.Game_ID == game,'n_passes_home1'] = n_passes.loc[(n_passes.Game_ID == game)&(n_passes.Team == train.loc[train.Game_ID == game,'Home Team'].values[0]) & (n_passes.Half == '1st half' ),0].values[0]
        train.loc[train.Game_ID == game,'n_passes_home2'] = n_passes.loc[(n_passes.Game_ID == game)&(n_passes.Team == train.loc[train.Game_ID == game,'Home Team'].values[0]) & (n_passes.Half == '2nd half' ),0].values[0]
        train.loc[train.Game_ID == game,'n_passes_away1'] = n_passes.loc[(n_passes.Game_ID == game)&(n_passes.Team == train.loc[train.Game_ID == game,'Away Team'].values[0]) & (n_passes.Half == '1st half' ),0].values[0]
        train.loc[train.Game_ID == game,'n_passes_away2'] = n_passes.loc[(n_passes.Game_ID == game)&(n_passes.Team == train.loc[train.Game_ID == game,'Away Team'].values[0]) & (n_passes.Half == '2nd half' ),0].values[0]
        train.loc[train.Game_ID == game,'n_shots_home1'] = n_shots.loc[(n_shots.Game_ID == game)&(n_shots.Team == train.loc[train.Game_ID == game,'Home Team'].values[0]) & (n_shots.Half == '1st half' ),0].values[0]
        train.loc[train.Game_ID == game,'n_shots_home2'] = n_shots.loc[(n_shots.Game_ID == game)&(n_shots.Team == train.loc[train.Game_ID == game,'Home Team'].values[0]) & (n_shots.Half == '2nd half' ),0].values[0]
        train.loc[train.Game_ID == game,'n_shots_away1'] = n_shots.loc[(n_shots.Game_ID == game)&(n_shots.Team == train.loc[train.Game_ID == game,'Away Team'].values[0]) & (n_shots.Half == '1st half' ),0].values[0]
        train.loc[train.Game_ID == game,'n_shots_away2'] = n_shots.loc[(n_shots.Game_ID == game)&(n_shots.Team == train.loc[train.Game_ID == game,'Away Team'].values[0]) & (n_shots.Half == '2nd half' ),0].values[0]
        train.loc[train.Game_ID == game,'accu_pass_home1'] = accu_pass.loc[(accu_pass.Game_ID == game)&(accu_pass.Team == train.loc[train.Game_ID == game,'Home Team'].values[0]) & (accu_pass.Half == '1st half' ),0].values[0]
        train.loc[train.Game_ID == game,'accu_pass_home2'] = accu_pass.loc[(accu_pass.Game_ID == game)&(accu_pass.Team == train.loc[train.Game_ID == game,'Home Team'].values[0]) & (accu_pass.Half == '2nd half' ),0].values[0]
        train.loc[train.Game_ID == game,'accu_pass_away1'] = accu_pass.loc[(accu_pass.Game_ID == game)&(accu_pass.Team == train.loc[train.Game_ID == game,'Away Team'].values[0]) & (accu_pass.Half == '1st half' ),0].values[0]
        train.loc[train.Game_ID == game,'accu_pass_away2'] = accu_pass.loc[(accu_pass.Game_ID == game)&(accu_pass.Team == train.loc[train.Game_ID == game,'Away Team'].values[0]) & (accu_pass.Half == '2nd half' ),0].values[0]
        train.loc[train.Game_ID == game,'accu_shots_home1'] = accu_shots.loc[(accu_shots.Game_ID == game)&(accu_shots.Team == train.loc[train.Game_ID == game,'Home Team'].values[0]) & (accu_shots.Half == '1st half' ),0].values[0]
        train.loc[train.Game_ID == game,'accu_shots_home2'] = accu_shots.loc[(accu_shots.Game_ID == game)&(accu_shots.Team == train.loc[train.Game_ID == game,'Home Team'].values[0]) & (accu_shots.Half == '2nd half' ),0].values[0]
        train.loc[train.Game_ID == game,'accu_shots_away1'] = accu_shots.loc[(accu_shots.Game_ID == game)&(accu_shots.Team == train.loc[train.Game_ID == game,'Away Team'].values[0]) & (accu_shots.Half == '1st half' ),0].values[0]
        train.loc[train.Game_ID == game,'accu_shots_away2'] = accu_shots.loc[(accu_shots.Game_ID == game)&(accu_shots.Team == train.loc[train.Game_ID == game,'Away Team'].values[0]) & (accu_shots.Half == '2nd half' ),0].values[0]
    test[['n_passes_home1','n_shots_home1','accu_pass_home1','accu_shots_home1',
          'n_passes_away1','n_shots_away1','accu_pass_away1','accu_shots_away1',
          'n_passes_home2','n_shots_home2','accu_pass_home2','accu_shots_home2',
          'n_passes_away2','n_shots_away2','accu_pass_away2','accu_shots_away2']]=0
    exep = []
    for game in tqdm(test.Game_ID):
        try:
            test.loc[test.Game_ID == game,'n_passes_home1'] = n_passes.loc[(n_passes.Game_ID == game)&(n_passes.Team == test.loc[test.Game_ID == game,'Home Team'].values[0]) & (n_passes.Half == '1st half' ),0].values[0]
        except:
            exep.append(game)
            continue
        try:
            test.loc[test.Game_ID == game,'n_passes_home2'] = n_passes.loc[(n_passes.Game_ID == game)&(n_passes.Team == test.loc[test.Game_ID == game,'Home Team'].values[0]) & (n_passes.Half == '2nd half' ),0].values[0]
        except:
            exep.append(game)
            continue

        try:
            test.loc[test.Game_ID == game,'n_passes_away1'] = n_passes.loc[(n_passes.Game_ID == game)&(n_passes.Team == test.loc[test.Game_ID == game,'Away Team'].values[0]) & (n_passes.Half == '1st half' ),0].values[0]
        except:
            exep.append(game)
            continue
        try:
            test.loc[test.Game_ID == game,'n_passes_away2'] = n_passes.loc[(n_passes.Game_ID == game)&(n_passes.Team == test.loc[test.Game_ID == game,'Away Team'].values[0]) & (n_passes.Half == '2nd half' ),0].values[0]
        except:
            exep.append(game)
            continue
        try:
            test.loc[test.Game_ID == game,'n_shots_home1'] = n_shots.loc[(n_shots.Game_ID == game)&(n_shots.Team == test.loc[test.Game_ID == game,'Home Team'].values[0]) & (n_shots.Half == '1st half' ),0].values[0]
        except:
            exep.append(game)
            continue
        try:
            test.loc[test.Game_ID == game,'n_shots_home2'] = n_shots.loc[(n_shots.Game_ID == game)&(n_shots.Team == test.loc[test.Game_ID == game,'Home Team'].values[0]) & (n_shots.Half == '2nd half' ),0].values[0]
        except:
            exep.append(game)
            continue
        try:
            test.loc[test.Game_ID == game,'n_shots_away1'] = n_shots.loc[(n_shots.Game_ID == game)&(n_shots.Team == test.loc[test.Game_ID == game,'Away Team'].values[0]) & (n_shots.Half == '1st half' ),0].values[0]
        except:
            exep.append(game)
            continue
        try:
            test.loc[test.Game_ID == game,'n_shots_away2'] = n_shots.loc[(n_shots.Game_ID == game)&(n_shots.Team == test.loc[test.Game_ID == game,'Away Team'].values[0]) & (n_shots.Half == '2nd half' ),0].values[0]
        except:
            exep.append(game)
            continue
        try:
            test.loc[test.Game_ID == game,'accu_pass_home1'] = accu_pass.loc[(accu_pass.Game_ID == game)&(accu_pass.Team == test.loc[test.Game_ID == game,'Home Team'].values[0]) & (accu_pass.Half == '1st half' ),0].values[0]
        except:
            exep.append(game)
            continue
        try:
            test.loc[test.Game_ID == game,'accu_pass_home2'] = accu_pass.loc[(accu_pass.Game_ID == game)&(accu_pass.Team == test.loc[test.Game_ID == game,'Home Team'].values[0]) & (accu_pass.Half == '2nd half' ),0].values[0]
        except:
            exep.append(game)
            continue
        try:
            test.loc[test.Game_ID == game,'accu_pass_away1'] = accu_pass.loc[(accu_pass.Game_ID == game)&(accu_pass.Team == test.loc[test.Game_ID == game,'Away Team'].values[0]) & (accu_pass.Half == '1st half' ),0].values[0]
        except:
            exep.append(game)
            continue
        try:
            test.loc[test.Game_ID == game,'accu_pass_away2'] = accu_pass.loc[(accu_pass.Game_ID == game)&(accu_pass.Team == test.loc[test.Game_ID == game,'Away Team'].values[0]) & (accu_pass.Half == '2nd half' ),0].values[0]
        except:
            exep.append(game)
            continue
        try:
            test.loc[test.Game_ID == game,'accu_shots_home1'] = accu_shots.loc[(accu_shots.Game_ID == game)&(accu_shots.Team == test.loc[test.Game_ID == game,'Home Team'].values[0]) & (accu_shots.Half == '1st half' ),0].values[0]
        except:
            exep.append(game)
            continue
        try:
            test.loc[test.Game_ID == game,'accu_shots_home2'] = accu_shots.loc[(accu_shots.Game_ID == game)&(accu_shots.Team == test.loc[test.Game_ID == game,'Home Team'].values[0]) & (accu_shots.Half == '2nd half' ),0].values[0]
        except:
            exep.append(game)
            continue
        try:
            test.loc[test.Game_ID == game,'accu_shots_away1'] = accu_shots.loc[(accu_shots.Game_ID == game)&(accu_shots.Team == test.loc[test.Game_ID == game,'Away Team'].values[0]) & (accu_shots.Half == '1st half' ),0].values[0]
        except:
            exep.append(game)
            continue
        try:
            test.loc[test.Game_ID == game,'accu_shots_away2'] = accu_shots.loc[(accu_shots.Game_ID == game)&(accu_shots.Team == test.loc[test.Game_ID == game,'Away Team'].values[0]) & (accu_shots.Half == '2nd half' ),0].values[0]
        except:
            exep.append(game)
            continue
    # train
# train[train.Game_ID == 'ID_00EFNL7L']
# train
#     th= 2
    test_stats = test_stats[test_stats.X<300]
    train_stats['dist_cent'] = np.sqrt((train_stats.X - 52.5)**2 + (train_stats.Y - 34)**2) 
    a = train_stats[train_stats['Accurate passes']==1].groupby(['Game_ID','Team']).apply(lambda x : (x['dist_cent']<th).sum())
    b = a.reset_index().copy()
    ch = train_stats.groupby(['Game_ID','Team']).apply(lambda x: x.Player_ID.unique().shape[0]%11)
    ch = ch.reset_index()
    change_mean_X = mean_X_change(train_stats)
    c =train_stats.groupby(['Game_ID','Team','End_minutes']).apply(lambda x: x['Start_minutes'].min()).reset_index()
    c =c.groupby(['Game_ID','Team']).apply(lambda x: (x['End_minutes']-x[0]).mean()).reset_index()
    train['n_changes_home'] = 0
    train['n_changes_away'] = 0
    train['mean_X_home'] = 0
    train['mean_X_away'] = 0
    train['change_mean_X_home'] = 0
    train['change_mean_X_away'] = 0
    train['sum_min_home'] = 0
    train['sum_min_away'] = 0
    mean_X = train_stats.groupby(['Game_ID','Team']).apply(lambda x : x['X'].mean()).reset_index()
    train['h_cent'] = 0
    train['a_cent'] = 0
    for g in tqdm(train.Game_ID):
        home = train.loc[train.Game_ID == g ,'Home Team'].values[0]
        away = train.loc[train.Game_ID == g ,'Away Team'].values[0]
        train.loc[train.Game_ID == g,'h_cent'] = b.loc[(b.Game_ID == g)&(b.Team == home),0].values[0]
        train.loc[train.Game_ID == g,'a_cent'] = b.loc[(b.Game_ID == g)&(b.Team == away),0].values[0]
        train.loc[train.Game_ID == g,'mean_X_home'] = mean_X.loc[(mean_X.Game_ID == g)&(mean_X.Team == home),0].values[0]
        train.loc[train.Game_ID == g,'mean_X_away'] = mean_X.loc[(mean_X.Game_ID == g)&(mean_X.Team == away),0].values[0]
        train.loc[train.Game_ID == g,'n_changes_home'] = ch.loc[(ch.Game_ID == g)&(ch.Team == home),0].values[0]
        train.loc[train.Game_ID == g,'n_changes_away'] = ch.loc[(ch.Game_ID == g)&(ch.Team == away),0].values[0]
        train.loc[train.Game_ID == g,'change_mean_X_home'] = change_mean_X.loc[(change_mean_X.Game_ID == g)&(change_mean_X.Team == home),'mean_change_X'].values[0]
        train.loc[train.Game_ID == g,'change_mean_X_away'] = change_mean_X.loc[(change_mean_X.Game_ID == g)&(change_mean_X.Team == away),'mean_change_X'].values[0]
        train.loc[train.Game_ID == g,'sum_min_home'] = c.loc[(c.Game_ID == g)&(c.Team == home),0].values[0]
        train.loc[train.Game_ID == g,'sum_min_away'] = c.loc[(c.Game_ID == g)&(c.Team == away),0].values[0]
    test_stats['dist_cent'] = np.sqrt((test_stats.X - 52.5)**2 + (test_stats.Y - 34)**2) 
    a = test_stats[test_stats['Accurate passes']==1].groupby(['Game_ID','Team']).apply(lambda x : (x['dist_cent']<th).sum())
    b = a.reset_index().copy()
    mean_X = test_stats.groupby(['Game_ID','Team']).apply(lambda x : x['X'].mean()).reset_index()
    ch = test_stats.groupby(['Game_ID','Team']).apply(lambda x: x.Player_ID.unique().shape[0]%11)
    ch = ch.reset_index()
    change_mean_X = mean_X_change(test_stats)
    c =test_stats.groupby(['Game_ID','Team','End_minutes']).apply(lambda x: x['Start_minutes'].min()).reset_index()
    c =c.groupby(['Game_ID','Team']).apply(lambda x: (x['End_minutes']-x[0]).mean()).reset_index()
    test['change_mean_X_home'] = 0
    test['change_mean_X_away'] = 0
    test['h_cent'] = 0
    test['a_cent'] = 0
    test['n_changes_home'] = 0
    test['n_changes_away'] = 0
    test['mean_X_home'] = 0
    test['mean_X_away'] = 0
    
    for g in tqdm(test.Game_ID):
        home = test.loc[test.Game_ID == g ,'Home Team'].values[0]
        away = test.loc[test.Game_ID == g ,'Away Team'].values[0]
        try:
            test.loc[test.Game_ID == g,'mean_X_home'] = mean_X.loc[(mean_X.Game_ID == g)&(mean_X.Team == home),0].values[0]
        except:
            continue
        try:
            test.loc[test.Game_ID == g,'mean_X_away'] = mean_X.loc[(mean_X.Game_ID == g)&(mean_X.Team == away),0].values[0]
        except:
            continue
        try:
            test.loc[test.Game_ID == g,'n_changes_home'] = ch.loc[(ch.Game_ID == g)&(ch.Team == home),0].values[0]
        except:
            continue
        try:
            test.loc[test.Game_ID == g,'n_changes_away'] = ch.loc[(ch.Game_ID == g)&(ch.Team == away),0].values[0]
        except:
            continue
        try:
            test.loc[test.Game_ID == g,'change_mean_X_home'] = change_mean_X.loc[(change_mean_X.Game_ID == g)&(change_mean_X.Team == home),'mean_change_X'].values[0]
        except:
            continue
        try:
            test.loc[test.Game_ID == g,'change_mean_X_away'] = change_mean_X.loc[(change_mean_X.Game_ID == g)&(change_mean_X.Team == away),'mean_change_X'].values[0]
        except:
            continue
        try:
            test.loc[test.Game_ID == g,'sum_min_home'] = c.loc[(c.Game_ID == g)&(c.Team == home),0].values[0]
        except:
            continue
        try:
            test.loc[test.Game_ID == g,'sum_min_away'] = c.loc[(c.Game_ID == g)&(c.Team == away),0].values[0]
        except:
            continue

        try:
            test.loc[test.Game_ID == g,'h_cent'] = b.loc[(b.Game_ID == g)&(b.Team == home),0].values[0]
        except:
            continue
        try:
            test.loc[test.Game_ID == g,'a_cent'] = b.loc[(b.Game_ID == g)&(b.Team == away),0].values[0]
        except:
            continue
    train.fillna(0,inplace = True)# grouped.loc[(grouped.Game_ID == game)]
    test.fillna(0,inplace = True)
    train_cols = train.columns.difference(['Date', 'Season','Match_ID', 'Game_ID','Score','Train','Home Team','Away Team'])
    if scale:
        from sklearn.preprocessing import MinMaxScaler
        sc = MinMaxScaler()
        scal = ['n_passes_away1',
        'n_passes_away2',       
        'n_passes_home1',        
        'n_passes_home2',        
        'n_shots_away1',
        'n_shots_away2',
        'n_shots_home1',
        'n_shots_home2']
        for s in scal:
            sc = MinMaxScaler()
            train[s] = sc.fit_transform(train[s].values.reshape(-1, 1))
            test[s] = sc.transform(test[s].values.reshape(-1, 1))

    return train[train_cols],test[train_cols],train['Score'],test['Score']
#     return train , test
        # test.loc[test.Game_ID =='ID_00EFNL7L']
            # train.loc[train.Game_ID =='ID_00EFNL7L']

In [5]:
le=LabelEncoder()
train["Score"] = le.fit_transform(train["Score"])
score_mapping = dict(zip(le.classes_, range(len(le.classes_))))

In [6]:
train.sort_values(by=['Date'],inplace=True)

In [45]:
from sklearn.model_selection import train_test_split
trai,val= train_test_split(train, test_size=0.2,shuffle=False)

In [46]:
pro_train_X,pro_test_X,pro_train_y,pro_test_y=preprocess(trai,val,
                                                         train_stats[train_stats.Game_ID.isin(trai.Game_ID)],
                                                         train_stats[train_stats.Game_ID.isin(val.Game_ID)])

In [47]:
X,test_data,y,_=preprocess(train,test,train_stats,test_stats)

In [51]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split #for split the data
from sklearn.metrics import accuracy_score  #for accuracy_score
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
from sklearn.metrics import confusion_matrix #for confusion matrix
import seaborn as sns

model= DecisionTreeClassifier(criterion='gini', 
                             min_samples_split=10,min_samples_leaf=1,
                             max_features='auto')
model.fit(pro_train_X,pro_train_y)
dt_pred=model.predict(pro_test_X)
kfold = KFold(n_splits=3) # k=10, split the data into 10 equal parts
result_tree=cross_val_score(model,pro_train_X,pro_train_y,cv=3,scoring='neg_log_loss')
print('The overall score for Decision Tree classifier is:',round(result_tree.mean(),2))
y_pred = cross_val_predict(model,pro_train_X,pro_train_y,cv=3)
sns.heatmap(confusion_matrix(dt_pred,pro_test_y),annot=True,cmap='summer')
plt.title('KNN Confusion_matrix')

In [52]:
#importing models
import lightgbm as lgb 
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,log_loss

In [53]:
lr=LogisticRegression(C=0.1,max_iter=1000)
lr.fit(pro_train_X,pro_train_y)
#Predictiing on Test Set
y_pred = lr.predict_proba(pro_test_X)
print(log_loss(pro_test_y,y_pred))

In [54]:
xgb=XGBClassifier(n_estimators=1000, max_depth=1, learning_rate=0.01,colsample_bytree=0.2,
                  random_state=12,subsample=0.8)
xgb.fit(pro_train_X,pro_train_y)
#Predictiing on Test Set
y_pred = xgb.predict_proba(pro_test_X)
print(log_loss(pro_test_y,y_pred))

In [55]:
clf = RandomForestClassifier(n_estimators=1200 , random_state=12,max_depth=2,max_features=0.6)
clf.fit(pro_train_X,pro_train_y)
#Predictiing on Test Set
y_pred = clf.predict_proba(pro_test_X)
print(log_loss(pro_test_y,y_pred))

In [56]:
lr.fit(X,y)
clf.fit(X,y)
xgb.fit(X,y)
test_predictions  = lr.predict_proba(test_data)
test_predictions_xgb = xgb.predict_proba(test_data)
test_predictions_rf = clf.predict_proba(test_data)

In [62]:
# test_data.isna().sum().sum()
# test_data.describe()
# X.isna().sum().sum()
# TARGET = "loss"
# ID = "id"
import numpy as np
import pandas as pd
import lightgbm as lgb
import gc

from sklearn.model_selection import StratifiedKFold,KFold
from sklearn import metrics
SEED = 42
NUM_CLASSES = 3
EARLY_STOPPING_ROUNDS = 30
N_ESTIMATORS = 100
K_FOLDS = 3
# Define Parameters for LGBM
lgb_params = {
    "objective": "multiclass",
    "boosting_type": "gbdt",
    "learning_rate": 0.01,
    "num_class": NUM_CLASSES,
    "num_leaves": 31,
    "tree_learner": "serial",
    "n_jobs": 4,
    "seed": SEED,
    "max_depth": -1,
#     "max_bin": 255,
    "metric": "multi_logloss",
    "verbose": -1,
}
train_X = X.copy()
train_Y = y.copy()
test_X = test_data.copy()
print(f"Shape of train_X : {train_X.shape}, test_X: {test_X.shape}, train_Y: {train_Y.shape}")

predictors = list(train_X.columns)
# print(f"List of features to be used {list(predictors)}")

# Selecting n_splits to be 3, since class 42 has 
# just 3 instances
kf = KFold(random_state=42,n_splits=K_FOLDS, shuffle=True)
y_oof = np.zeros(shape=(len(train_X), NUM_CLASSES))
y_predicted = np.zeros(shape=(len(test_X), NUM_CLASSES))
cv_scores = []


from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
def _get_X_Y_DF_from_CV(train_X, train_Y, train_index, validation_index):
    X_train, X_validation = (
        train_X.iloc[train_index],
        train_X.iloc[validation_index],
    )
    y_train, y_validation = (
        train_Y.iloc[train_index],
        train_Y.iloc[validation_index],
    )
    return X_train, X_validation, y_train, y_validation
fold = 0
n_folds = kf.get_n_splits()
for train_index, validation_index in kf.split(X=train_X, y=train_Y):
    fold += 1
    print(f"fold {fold} of {n_folds}")

    X_train, X_validation, y_train, y_validation = _get_X_Y_DF_from_CV(
        train_X, train_Y, train_index, validation_index
    )

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_validation, y_validation, reference=lgb_train)

    model = lgb.train(
        lgb_params,
        lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        verbose_eval=10,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        num_boost_round=N_ESTIMATORS,
        feature_name=predictors,
        categorical_feature="auto",
    )
    del lgb_train, lgb_eval, train_index, X_train, y_train
    gc.collect()

    y_oof[validation_index] = model.predict(
        X_validation, num_iteration=model.best_iteration
    )

    y_predicted += model.predict(
        test_X.values, num_iteration=model.best_iteration
    )

    best_iteration = model.best_iteration
    print(f"Best number of iterations for fold {fold} is: {best_iteration}")

    cv_oof_score = metrics.log_loss(y_validation, y_oof[validation_index])
    cv_scores.append(cv_oof_score)
    print(f"CV OOF Score for fold {fold} is {cv_oof_score}")

    del validation_index, X_validation, y_validation
    gc.collect()

y_predicted /= n_folds
oof_score = round(metrics.log_loss(train_Y, y_oof), 5)
avg_cv_scores = round(sum(cv_scores) / len(cv_scores), 5)
std_cv_scores = round(np.array(cv_scores).std(), 5)

In [ ]:
X_train

In [ ]:
# test_predictions
# X['target'] = y
# test_data
# y_predicted
# X['h_cent']
# X['score'] = y
# train_all = X.copy()
# train_all['score'] = y

In [ ]:
# ((train_all.loc[train_all.score ==2,'h_cent'] - train_all.loc[train_all.score ==2,'a_cent'])<0).sum()# .apply(lambda x: x.a_cent - x.h_cent)
# sns.distplot(train_all.loc[train_all.score ==2,'mean_X_home'])
# sns.distplot(train_all.loc[train_all.score ==2,'mean_X_away'])

# train_all

In [ ]:
# X.corr()['target'].sort_values(ascending = False,key = abs)

In [66]:
Test = test.copy()
cols=le.inverse_transform([*range(3)])
Test[cols]= y_predicted

In [69]:
submit = Test[["Game_ID",'Away win', 'Draw', 'Home Win']]
submit.drop_duplicates(subset = ["Game_ID"], inplace=True)
submit = submit.reset_index(drop=True)
submit.to_csv("submission_gbm10.csv", index=False)

In [70]:
submit

In [61]:
# test
# submit
# ids = ['ID_QKQ00O16',
# 'ID_172V2IXW',
# 'ID_2ZK5TE39',
# 'ID_IS54QGW9',
# 'ID_YG9IONLS']
# X.Train
# ids
# test_stats[test_stats.Game_ID == ids[4]]
# X.shape
X.info()

In [ ]:
submit.iloc[0][['Away win', 'Draw', 'Home Win']]

In [ ]:
submit2 = submit.copy()
submit2[['Away win', 'Draw', 'Home Win']] = submit2[['Away win', 'Draw', 'Home Win']].round()

In [ ]:
a = submit2[['Away win', 'Draw', 'Home Win']].sum(axis = 1) == 0
submit2.loc[a,['Away win', 'Draw', 'Home Win']] = submit.loc[a,['Away win', 'Draw', 'Home Win']] 

In [ ]:
# submit2.max(axis = 1).idxmin()
submit2.to_csv("submission_lr5.csv", index=False)

In [ ]:
submit.max(axis =1).max()